<a href="https://colab.research.google.com/github/KimDukJung/bbc/blob/main/translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1번 블록 : 패키지 임포트

import numpy as np
import pandas as pd
from time import time
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# 하이퍼파라미터 설정
MY_HIDDEN = 128
MY_EPOCH = 500

In [ ]:
# 2번 블록 : 데이터 로딩

# 구글 드라이브 파일 경로
path = '/content/drive/MyDrive/dataset/translate.csv'

# 영어-한글 번역 데이터 읽기
raw = pd.read_csv(path, header=None, encoding='utf-8')
eng_kor = raw.values.tolist()

# 영어-한글 번역 데이터 통계 출력
print('영어-한글 번역 데이터 수 : ', len(eng_kor))
print('영어-한글 번역 데이터\n', eng_kor)


영어-한글 번역 데이터 수 :  110
영어-한글 번역 데이터
 [['cold', '감기'], ['come', '오다'], ['cook', '요리'], ['copy', '복사'], ['cost', '비용'], ['date', '날짜'], ['deal', '거래'], ['deep', '깊은'], ['desk', '책상'], ['down', '아래'], ['dust', '먼지'], ['duty', '의무'], ['each', '각각'], ['east', '동쪽'], ['easy', '쉽다'], ['exit', '탈출'], ['face', '얼굴'], ['fact', '사실'], ['fall', '가을'], ['farm', '농장'], ['feet', '다리'], ['find', '찾다'], ['fine', '좋다'], ['fish', '생선'], ['flow', '흐름'], ['fund', '기금'], ['gain', '수익'], ['game', '놀이'], ['gift', '선물'], ['girl', '소녀'], ['give', '주다'], ['goal', '목적'], ['gray', '회색'], ['hair', '머리'], ['harm', '피해'], ['hell', '지옥'], ['help', '도움'], ['high', '높은'], ['hole', '구멍'], ['hope', '소망'], ['hour', '시각'], ['join', '합류'], ['kiss', '키스'], ['knee', '무릎'], ['lady', '부인'], ['late', '늦은'], ['left', '왼쪽'], ['life', '생명'], ['loss', '손해'], ['love', '사랑'], ['luck', '행운'], ['mail', '우편'], ['male', '남자'], ['many', '많은'], ['meal', '식사'], ['meat', '고기'], ['menu', '메뉴'], ['milk', '우유'], ['name', '이름'], ['news', '뉴스'], ['n

In [ ]:
# 3번 블록 : 알파벳 사전 만들기
# 알파벳 사전 구성 => SEPabcd.......+ 한글 글자

e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
print(e_alpha)

# 한글 단어데이터 로딩
path = '/content/drive/MyDrive/dataset/korean.csv'
k_alpha = pd.read_csv(path, header=None, encoding='utf-8')
k_alpha = k_alpha[0].values.tolist()
print(k_alpha)

['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']


In [ ]:
# 4번 블록 : 전체 알파벳 리스트

alphabet = e_alpha + k_alpha
# print(alphabet)

# 알파벳 사전 만들기
# 알파벳을 고유 숫자로 전환

char_to_num = dict((c, i) for i, c in enumerate(alphabet))
print(char_to_num)
print('총 알파벳 수 : ', len(alphabet))
print('영어 알파벳 수 :',len(e_alpha))
print('한글 알파벳 수 :',len(k_alpha))
n_input = len(alphabet)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [ ]:
# 5번 블록

# 영어-한글 번역 데이터 인코딩 작업
# 인코더 입력값 : 영어 단어를 단어사전에서 숫자로 변경한 다음에 원핫 인코딩
# 디코더 입력값 : 'S' + 한글 정답 데이터를 단어사전에서 숫자로 변경한 다음 원핫 인코딩
# 디코더 출력값 : 한글 단어 + 'E' 를 정수 인코딩

def encode(eng_kor):
  enc_in = []
  dec_in = []
  rnn_out = []
  for data in eng_kor:
    # 인코더 입력값 인코딩
    #print(data[0])
    eng = [char_to_num[c] for c in data[0]]
    #print(eng)
    one_hot = np.eye(n_input)[eng]
    #print(one_hot)
    enc_in.append(one_hot)
    # 디코더 입력값 인코딩
    #print(data[1])
    kor = [char_to_num[c] for c in ('S' +  data[1])]
    #print(kor)
    one_hot = np.eye(n_input)[kor]
    #print(one_hot)
    dec_in.append(one_hot)
    # 디코더 출력값 인코딩
    target = [char_to_num[c] for c in (data[1] + 'E')]
    #print(target)
    rnn_out.append(target)

  return enc_in, dec_in, rnn_out

# encode([['wood','나무']])

In [ ]:
# 6번 블록

# 샘플 번역 데이터 인코딩
sample = [['word','단어']]
enc_in, dec_in, rnn_out = encode(sample)

print('인코더 입력값 : ', enc_in)
print('디코더 입력값 : ', dec_in)
print('디코더 출력값 : ', rnn_out)

인코더 입력값 :  [array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
# 7번 블록

# 영어-한글 번역 데이터 전체를 인코딩 처리
X_enc, X_dec, Y_dec = encode(eng_kor)
# print('encode 함수의 출력 타입 : ',type(X_enc))
# 파이썬 리스트를 Numpy의 array로 전환
X_enc = np.array(X_enc)
X_dec = np.array(X_dec)
Y_dec = np.array(Y_dec)

# Numpy array 모양 출력(shape)
print('X_enc.shape : ', X_enc.shape)
print('X_dec.shape : ', X_dec.shape)
print('Y_dec.shape : ', Y_dec.shape)

Y_dec = np.expand_dims(Y_dec, axis=2)
print('Y_dec.shape : ', Y_dec.shape)

X_enc.shape :  (110, 4, 171)
X_dec.shape :  (110, 3, 171)
Y_dec.shape :  (110, 3)
Y_dec.shape :  (110, 3, 1)


In [ ]:
# 8번 블록

# 인코더 구현
ENC_IN = Input(shape=(4, n_input))

_, State_h, State_c = LSTM(units = MY_HIDDEN, return_state = True)(ENC_IN)


# 인코더와 디코더 연결 ( 셀 스테이트, 히든 스테이트)
link = [State_h, State_c]

print(link)

[<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_2>, <KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_3>]


In [ ]:
# 9번 블록

# 디코더 구현
DEC_IN = Input(shape=(3, n_input))
DEC_LSTM = LSTM(units = MY_HIDDEN, return_sequences = True)(DEC_IN, initial_state = link)

#최종 출력층
DEC_OUT = Dense(units = n_input, activation = 'softmax')(DEC_LSTM)

# 전체 RNN 모델
model = Model(inputs=[ENC_IN, DEC_IN], outputs = DEC_OUT)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 4, 171)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 3, 171)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 128), (None,   │        153,600 │ input_layer[0][0]      │
│                           │ 128), (None, 128)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 3, 128)         │        153,600 │ input_layer_1[0][0],   │
│                           │                        │                │ lstm[0][1], lstm[0][2] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 3, 171)         │         22,059 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 329,259 (1.26 MB)

 Trainable params: 329,259 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 10번 블록

# RNN 학습 환경 설정
model.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy')

print('학습 시작')
begin = time()

model.fit([X_enc, X_dec],
          Y_dec,
          epochs = MY_EPOCH,
          verbose=0)

end = time()
print('학습 시간 : {:.2f}초'.format(end - begin))

학습 시작
학습 시간 : 26.99초


In [ ]:
# 11번 블록

# 임의의 5개 영어-한글 데이터 선택
pick = np.random.randint(0 , len(eng_kor), 5)
easy = [eng_kor[i] for i in pick]
# print(easy)

for i in range(5):
  easy[i][1] = 'PP'

print(easy)

# 새 번역 데이터 추가
hard = [['love','PP'],['olve','PP'],['lvoe','PP'],['loev','PP'],['eovl','PP']]

[['gift', 'PP'], ['wind', 'PP'], ['hole', 'PP'], ['tiny', 'PP'], ['find', 'PP']]


In [ ]:
# 12번 블록

# 전체 샘플 버역 데이터 합성
new_data = easy + hard
print(new_data)

# 샘플 번역 데이터 인코딩 처리
enc_in, dec_in, rnn_out = encode(new_data)

enc_in = np.array(enc_in)
dec_in = np.array(dec_in)

# Numpy array 모양 출력(shape)
print('enc_in.shape : ', enc_in.shape)
print('dec_in.shape : ', dec_in.shape)

# RNN 으로 예측
pred = model.predict([enc_in, dec_in])
print('번역 결과')

for i in range(len(new_data)):
  eng = new_data[i][0]

  # 171개의 softmax 출력 확률에서 최대값 위치 찾기
  word = np.argmax(pred[i], axis=1)

  # 한글 한 글자씩 추출
  kor =''
  for j in range(2):
    kor = kor + alphabet[word[j]]

  # 번역 결과 출력

  print(eng, ' : ', kor)

[['gift', 'PP'], ['wind', 'PP'], ['hole', 'PP'], ['tiny', 'PP'], ['find', 'PP'], ['love', 'PP'], ['olve', 'PP'], ['lvoe', 'PP'], ['loev', 'PP'], ['eovl', 'PP']]
enc_in.shape :  (10, 4, 171)
dec_in.shape :  (10, 3, 171)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
번역 결과
gift  :  선물
wind  :  바람
hole  :  구멍
tiny  :  작은
find  :  찾다
love  :  사랑
olve  :  사랑
lvoe  :  사랑
loev  :  사랑
eovl  :  도다
